In [2]:
import csv
import lzstring
from collections import namedtuple, Counter
import json
from memoize import memoize
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
import sys
import json

In [3]:
csv.field_size_limit(sys.maxsize)

# [num unique urls, num unique urls typed, total visits, total typed, first visit time, last visit time]
domaininfo = namedtuple('domaininfo', ['num_unique_urls', 'num_unique_urls_typed', 'total_visits', 'total_typed', 'first_visit_time', 'last_visit_time'])

decompressFromEncodedURIComponent = lzstring.LZString().decompressFromEncodedURIComponent

#filepath = 'difficultyselectionexp_may31_11am.csv'
filepath = 'difficultyselectionexp_june25_9pm.csv'
reader = csv.DictReader(open(filepath))

def extract_domain_visit_info(domain_visit_info_compressed):
  domain_visit_info = json.loads(decompressFromEncodedURIComponent(domain_visit_info_compressed))
  output = {}
  for k,v in domain_visit_info.items():
    linedata = domaininfo(*v)
    output[k] = linedata
  return output

alldata = []

for alldata_item in reader:
  if alldata_item['selected_difficulty'] not in ['nothing', 'easy', 'medium', 'hard']:
    continue
  if alldata_item['domain_visit_info_compressed'] == None or len(alldata_item['domain_visit_info_compressed']) == 0:
    continue
  alldata_item['domain_visit_info'] = extract_domain_visit_info(alldata_item['domain_visit_info_compressed'])
  alldata.append(alldata_item)


In [74]:
np.random.shuffle(alldata)
training_data = alldata[:round(len(alldata)*0.8)]
test_data = alldata[round(len(alldata)*0.8):]
print(len(training_data))
print(len(test_data))

346
86


In [75]:
def extract_labels_alldata(data):
  return np.array([line['selected_difficulty'] for line in data])

@memoize
def get_most_common_label():
  label_to_count = Counter()
  for line in training_data:
    label = line['selected_difficulty']
    label_to_count[label] += 1
  sorted_by_count = sorted(label_to_count.items(), key=lambda x: x[1], reverse=True)
  return sorted_by_count[0][0]

@memoize
def get_most_visited_domains():
  domain_to_num_visits = Counter()
  for line in training_data:
    domain_visit_info = line['domain_visit_info']
    for domain,info in domain_visit_info.items():
      domain_to_num_visits[domain] += info.total_visits
  sorted_by_num_visits = sorted(domain_to_num_visits.items(), key=lambda x: x[1], reverse=True)
  return [x[0] for x in sorted_by_num_visits[:100]]

cnt = 0
@memoize
def get_most_common_domains():
  domain_to_num_visits = Counter()
  for line in training_data:
    domain_visit_info = line['domain_visit_info']
    for domain,info in domain_visit_info.items():
      domain_to_num_visits[domain] += 1
  sorted_by_num_visits = sorted(domain_to_num_visits.items(), key=lambda x: x[1], reverse=True)
  return [x[0] for x in sorted_by_num_visits[:100]]

def get_all_domains():
  domain_to_num_visits = Counter()
  for line in training_data:
    domain_visit_info = line['domain_visit_info']
    for domain,info in domain_visit_info.items():
      domain_to_num_visits[domain] += 1
  sorted_by_num_visits = sorted(domain_to_num_visits.items(), key=lambda x: x[1], reverse=True)
  return [x[0] for x in sorted_by_num_visits]



def get_num_visits_for_domain(domain_visit_info, domain):
  info = domain_visit_info.get(domain, None)
  if info != None:
    return info.total_visits
  return 0

def get_productivity():
    with open ('domain_to_productivity.json') as json_file:
        data = json.load(json_file)
        return data

domain_to_productivity = get_productivity()

def extract_features_for_user(domain_visit_info):
  cnt = 0
  domains = get_all_domains()
  final_features = [0, 0, 0, 0, 0];
  for x in domains:
        if x in domain_to_productivity.keys():
            final_features[domain_to_productivity[x] + 2] += get_num_visits_for_domain(domain_visit_info, x)
    
  #np.array([get_num_visits_for_domain(domain_visit_info, x) for x in domains])
  if np.sum(final_features) >= 1:
    final_features = np.divide(final_features, np.sum(final_features))
  return final_features

def extract_features_alldata(data):
  output = []
  for line in data:
    domain_visit_info = line['domain_visit_info']
    features = extract_features_for_user(domain_visit_info)
    output.append(features)
  return np.array(output)


In [76]:
def get_percent_correct(predicted_labels, actual_labels):
  if len(predicted_labels) != len(actual_labels):
    raise 'need predicted and actual labels to have same lengths'
  total = len(actual_labels)
  correct = 0
  for p,a in zip(predicted_labels, actual_labels):
    if p == a:
      correct += 1
  return correct / total

def test_baseline_classifier():
  most_common_label = get_most_common_label()
  predictions = [most_common_label for line in test_data]
  actual = extract_labels_alldata(test_data)
  percent_correct = get_percent_correct(predictions, actual)
  print('baseline classifier accuracy:', percent_correct)

def test_classifier(clf, str=None):
  actual = extract_labels_alldata(test_data)
  features_test = extract_features_alldata(test_data)
  predictions = clf.predict(features_test)
  percent_correct = get_percent_correct(predictions, actual)
  print(str + ' classifier testing accuracy:', percent_correct)

def training_error_classifier(clf, str=None):
  actual = extract_labels_alldata(training_data)
  features_train = extract_features_alldata(training_data)
  predictions = clf.predict(features_train)
  percent_correct = get_percent_correct(predictions, actual)
  print(str + ' classifier training accuracy:', round(percent_correct, 2))

def to_int_categorical(dt):
  # {'easy', 'hard', 'medium', 'nothing'}
  cat_dt = []
  for item in dt:
    if item == 'nothing':
      cat_dt.append(0)
    elif item == 'easy':
      cat_dt.append(1)
    elif item == 'medium':
      cat_dt.append(2)
    else:
      cat_dt.append(3)
  return np.array(cat_dt)
    
test_baseline_classifier()

baseline classifier accuracy: 0.4186046511627907


In [77]:
labels_train = extract_labels_alldata(training_data)
features_train = extract_features_alldata(training_data)

labels_train = np.delete(labels_train, 2, 0)
features_train = np.delete(features_train, 2, 0)

clf = RandomForestClassifier(max_depth=10, random_state=0)
clf.fit(features_train, labels_train)
test_classifier(clf, 'RF')

clf = KNeighborsClassifier(n_neighbors=3, p=1)
clf.fit(features_train, labels_train)
test_classifier(clf, 'KNN')

RF classifier testing accuracy: 0.36046511627906974
KNN classifier testing accuracy: 0.4883720930232558


In [78]:
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation
from keras.optimizers import SGD
from keras.utils import to_categorical
from keras import regularizers

labels_test = to_categorical(to_int_categorical(extract_labels_alldata(test_data)), num_classes=4)
features_test = extract_features_alldata(test_data)

labels_train = to_categorical(to_int_categorical(extract_labels_alldata(training_data)), num_classes=4)
features_train = extract_features_alldata(training_data)

labels_train = np.delete(labels_train, 2, 0)
features_train = np.delete(features_train, 2, 0)

In [97]:
model = Sequential()

model.add(Dense(40, activation='relu', input_dim=5, kernel_regularizer=regularizers.l2(0.0001)))
#model.add(Dropout(0.1))
model.add(Dense(100, activation='relu', kernel_regularizer=regularizers.l2(0.0001)))
#model.add(Dropout(0.1))
model.add(Dense(50, activation='relu', kernel_regularizer=regularizers.l2(0.0001)))
#model.add(Dropout(0.1))
model.add(Dense(4, activation='softmax', kernel_regularizer=regularizers.l2(0.0001)))

sgd = SGD(lr=0.06, decay=1e-3, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy',
              optimizer=sgd,
              metrics=['accuracy'])

model.fit(features_train, labels_train,
          epochs=1000,
          batch_size=32)
score = model.evaluate(features_test, labels_test, batch_size=32)

predictions = model.predict(features_test, batch_size=32)

print(predictions)
print(labels_test)
print(score)

Epoch 1/1000
345/345 [==============================] - 1s 3ms/step - loss: 1.3459 - acc: 0.3884
Epoch 2/1000
345/345 [==============================] - 0s 62us/step - loss: 1.3191 - acc: 0.4116
Epoch 3/1000
345/345 [==============================] - 0s 61us/step - loss: 1.3081 - acc: 0.4116
Epoch 4/1000
345/345 [==============================] - 0s 49us/step - loss: 1.3007 - acc: 0.4232
Epoch 5/1000
345/345 [==============================] - 0s 63us/step - loss: 1.2971 - acc: 0.4261
Epoch 6/1000
345/345 [==============================] - 0s 62us/step - loss: 1.2891 - acc: 0.4232
Epoch 7/1000
345/345 [==============================] - 0s 57us/step - loss: 1.2822 - acc: 0.4319
Epoch 8/1000
345/345 [==============================] - 0s 69us/step - loss: 1.2913 - acc: 0.4174
Epoch 9/1000
345/345 [==============================] - 0s 66us/step - loss: 1.2862 - acc: 0.4319
Epoch 10/1000
345/345 [==============================] - 0s 56us/step - loss: 1.2791 - acc: 0.4261
Epoch 11/1000
345/34

345/345 [==============================] - 0s 65us/step - loss: 1.2488 - acc: 0.4464
Epoch 84/1000
345/345 [==============================] - 0s 57us/step - loss: 1.2485 - acc: 0.4377
Epoch 85/1000
345/345 [==============================] - 0s 59us/step - loss: 1.2506 - acc: 0.4435
Epoch 86/1000
345/345 [==============================] - 0s 59us/step - loss: 1.2463 - acc: 0.4406
Epoch 87/1000
345/345 [==============================] - 0s 70us/step - loss: 1.2466 - acc: 0.4493
Epoch 88/1000
345/345 [==============================] - 0s 63us/step - loss: 1.2432 - acc: 0.4319
Epoch 89/1000
345/345 [==============================] - 0s 62us/step - loss: 1.2484 - acc: 0.4406
Epoch 90/1000
345/345 [==============================] - 0s 59us/step - loss: 1.2490 - acc: 0.4406
Epoch 91/1000
345/345 [==============================] - 0s 60us/step - loss: 1.2465 - acc: 0.4319
Epoch 92/1000
345/345 [==============================] - 0s 54us/step - loss: 1.2496 - acc: 0.4377
Epoch 93/1000
345/345 [=

345/345 [==============================] - 0s 62us/step - loss: 1.2218 - acc: 0.4522
Epoch 166/1000
345/345 [==============================] - 0s 54us/step - loss: 1.2209 - acc: 0.4493
Epoch 167/1000
345/345 [==============================] - 0s 59us/step - loss: 1.2216 - acc: 0.4464
Epoch 168/1000
345/345 [==============================] - 0s 57us/step - loss: 1.2188 - acc: 0.4609
Epoch 169/1000
345/345 [==============================] - 0s 61us/step - loss: 1.2249 - acc: 0.4551
Epoch 170/1000
345/345 [==============================] - 0s 57us/step - loss: 1.2207 - acc: 0.4493
Epoch 171/1000
345/345 [==============================] - 0s 57us/step - loss: 1.2203 - acc: 0.4406
Epoch 172/1000
345/345 [==============================] - 0s 57us/step - loss: 1.2157 - acc: 0.4725
Epoch 173/1000
345/345 [==============================] - 0s 60us/step - loss: 1.2404 - acc: 0.4464
Epoch 174/1000
345/345 [==============================] - 0s 58us/step - loss: 1.2186 - acc: 0.4406
Epoch 175/1000


345/345 [==============================] - 0s 54us/step - loss: 1.1993 - acc: 0.4725
Epoch 248/1000
345/345 [==============================] - 0s 51us/step - loss: 1.1949 - acc: 0.4696
Epoch 249/1000
345/345 [==============================] - 0s 49us/step - loss: 1.1976 - acc: 0.4551
Epoch 250/1000
345/345 [==============================] - 0s 49us/step - loss: 1.2044 - acc: 0.4667
Epoch 251/1000
345/345 [==============================] - 0s 52us/step - loss: 1.2016 - acc: 0.4783
Epoch 252/1000
345/345 [==============================] - 0s 49us/step - loss: 1.2029 - acc: 0.4638
Epoch 253/1000
345/345 [==============================] - 0s 50us/step - loss: 1.2073 - acc: 0.4551
Epoch 254/1000
345/345 [==============================] - 0s 51us/step - loss: 1.1994 - acc: 0.4725
Epoch 255/1000
345/345 [==============================] - 0s 55us/step - loss: 1.2026 - acc: 0.4609
Epoch 256/1000
345/345 [==============================] - 0s 63us/step - loss: 1.1936 - acc: 0.4754
Epoch 257/1000


345/345 [==============================] - 0s 69us/step - loss: 1.1811 - acc: 0.4783
Epoch 330/1000
345/345 [==============================] - 0s 63us/step - loss: 1.1730 - acc: 0.4812
Epoch 331/1000
345/345 [==============================] - 0s 69us/step - loss: 1.1858 - acc: 0.4928
Epoch 332/1000
345/345 [==============================] - 0s 68us/step - loss: 1.1919 - acc: 0.5130
Epoch 333/1000
345/345 [==============================] - 0s 70us/step - loss: 1.1678 - acc: 0.5014
Epoch 334/1000
345/345 [==============================] - 0s 75us/step - loss: 1.1704 - acc: 0.4812
Epoch 335/1000
345/345 [==============================] - 0s 63us/step - loss: 1.1887 - acc: 0.4725
Epoch 336/1000
345/345 [==============================] - 0s 56us/step - loss: 1.1741 - acc: 0.4899
Epoch 337/1000
345/345 [==============================] - 0s 64us/step - loss: 1.1730 - acc: 0.4899
Epoch 338/1000
345/345 [==============================] - 0s 58us/step - loss: 1.1756 - acc: 0.4783
Epoch 339/1000


...

345/345 [==============================] - 0s 54us/step - loss: 1.0830 - acc: 0.5449
Epoch 658/1000
345/345 [==============================] - 0s 49us/step - loss: 1.0804 - acc: 0.5449
Epoch 659/1000
345/345 [==============================] - 0s 55us/step - loss: 1.0870 - acc: 0.5536
Epoch 660/1000
345/345 [==============================] - 0s 55us/step - loss: 1.0838 - acc: 0.5507
Epoch 661/1000
345/345 [==============================] - 0s 52us/step - loss: 1.0724 - acc: 0.5565
Epoch 662/1000
345/345 [==============================] - 0s 57us/step - loss: 1.0834 - acc: 0.5623
Epoch 663/1000
345/345 [==============================] - 0s 51us/step - loss: 1.0962 - acc: 0.5449
Epoch 664/1000
345/345 [==============================] - 0s 56us/step - loss: 1.0700 - acc: 0.5478
Epoch 665/1000
345/345 [==============================] - 0s 51us/step - loss: 1.0750 - acc: 0.5478
Epoch 666/1000
345/345 [==============================] - 0s 55us/step - loss: 1.0852 - acc: 0.5652
Epoch 667/1000


345/345 [==============================] - 0s 68us/step - loss: 1.0640 - acc: 0.5681
Epoch 740/1000
345/345 [==============================] - 0s 62us/step - loss: 1.0566 - acc: 0.5768
Epoch 741/1000
345/345 [==============================] - 0s 68us/step - loss: 1.0848 - acc: 0.5507
Epoch 742/1000
345/345 [==============================] - 0s 58us/step - loss: 1.0698 - acc: 0.5391
Epoch 743/1000
345/345 [==============================] - 0s 56us/step - loss: 1.0592 - acc: 0.5420
Epoch 744/1000
345/345 [==============================] - 0s 54us/step - loss: 1.0848 - acc: 0.5391
Epoch 745/1000
345/345 [==============================] - 0s 56us/step - loss: 1.0594 - acc: 0.5507
Epoch 746/1000
345/345 [==============================] - 0s 59us/step - loss: 1.0738 - acc: 0.5449
Epoch 747/1000
345/345 [==============================] - 0s 57us/step - loss: 1.0663 - acc: 0.5594
Epoch 748/1000
345/345 [==============================] - 0s 50us/step - loss: 1.0464 - acc: 0.5681
Epoch 749/1000


345/345 [==============================] - 0s 58us/step - loss: 1.0430 - acc: 0.5652
Epoch 822/1000
345/345 [==============================] - 0s 51us/step - loss: 1.0319 - acc: 0.5507
Epoch 823/1000
345/345 [==============================] - 0s 58us/step - loss: 1.0410 - acc: 0.5826
Epoch 824/1000
345/345 [==============================] - 0s 64us/step - loss: 1.0285 - acc: 0.5594
Epoch 825/1000
345/345 [==============================] - 0s 58us/step - loss: 1.0327 - acc: 0.5652
Epoch 826/1000
345/345 [==============================] - 0s 59us/step - loss: 1.0336 - acc: 0.5478
Epoch 827/1000
345/345 [==============================] - 0s 53us/step - loss: 1.0265 - acc: 0.5768
Epoch 828/1000
345/345 [==============================] - 0s 57us/step - loss: 1.0572 - acc: 0.5507
Epoch 829/1000
345/345 [==============================] - 0s 62us/step - loss: 1.0442 - acc: 0.5623
Epoch 830/1000
345/345 [==============================] - 0s 57us/step - loss: 1.0442 - acc: 0.5681
Epoch 831/1000


345/345 [==============================] - 0s 52us/step - loss: 1.0080 - acc: 0.5710
Epoch 904/1000
345/345 [==============================] - 0s 54us/step - loss: 1.0156 - acc: 0.6000
Epoch 905/1000
345/345 [==============================] - 0s 55us/step - loss: 1.0226 - acc: 0.5710
Epoch 906/1000
345/345 [==============================] - 0s 54us/step - loss: 1.0088 - acc: 0.5826
Epoch 907/1000
345/345 [==============================] - 0s 51us/step - loss: 1.0078 - acc: 0.5797
Epoch 908/1000
345/345 [==============================] - 0s 53us/step - loss: 1.0054 - acc: 0.5739
Epoch 909/1000
345/345 [==============================] - 0s 55us/step - loss: 1.0140 - acc: 0.5623
Epoch 910/1000
345/345 [==============================] - 0s 54us/step - loss: 1.0063 - acc: 0.5855
Epoch 911/1000
345/345 [==============================] - 0s 53us/step - loss: 1.0030 - acc: 0.5652
Epoch 912/1000
345/345 [==============================] - 0s 53us/step - loss: 1.0209 - acc: 0.5478
Epoch 913/1000


345/345 [==============================] - 0s 63us/step - loss: 0.9888 - acc: 0.5913
Epoch 986/1000
345/345 [==============================] - 0s 62us/step - loss: 0.9713 - acc: 0.5855
Epoch 987/1000
345/345 [==============================] - 0s 59us/step - loss: 0.9748 - acc: 0.6000
Epoch 988/1000
345/345 [==============================] - 0s 54us/step - loss: 0.9839 - acc: 0.5826
Epoch 989/1000
345/345 [==============================] - 0s 61us/step - loss: 0.9879 - acc: 0.5855
Epoch 990/1000
345/345 [==============================] - 0s 47us/step - loss: 0.9573 - acc: 0.5884
Epoch 991/1000
345/345 [==============================] - 0s 51us/step - loss: 0.9675 - acc: 0.6174
Epoch 992/1000
345/345 [==============================] - 0s 56us/step - loss: 0.9722 - acc: 0.6029
Epoch 993/1000
345/345 [==============================] - 0s 49us/step - loss: 0.9641 - acc: 0.6087
Epoch 994/1000
345/345 [==============================] - 0s 52us/step - loss: 0.9617 - acc: 0.6145
Epoch 995/1000
